In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from google.colab import drive

In [2]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
def load_data():
    df = pd.read_csv('/content/drive/MyDrive/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
    print(f"Dataset shape: {df.shape}")
    return df

print(df['storey_range'])

0         10 TO 12
1         01 TO 03
2         01 TO 03
3         04 TO 06
4         01 TO 03
            ...   
196681    07 TO 09
196682    07 TO 09
196683    04 TO 06
196684    10 TO 12
196685    07 TO 09
Name: storey_range, Length: 196686, dtype: object


In [9]:
def clean_data(df):
    # Drop any duplicates
    df = df.drop_duplicates()

    # Convert month to datetime
    df['month'] = pd.to_datetime(df['month'])

    # Calculate flat age
    df['flat_age'] = df['month'].dt.year - df['lease_commence_date']

    # Extract numeric values from storey_range
    df['storey_median'] = df['storey_range'].apply(lambda x: sum(map(int, x.split(' TO '))) / 2)

    # Remove outliers using IQR method for resale_price
    Q1 = df['resale_price'].quantile(0.25)
    Q3 = df['resale_price'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df['resale_price'] < (Q1 - 1.5 * IQR)) | (df['resale_price'] > (Q3 + 1.5 * IQR)))]

    return df
def feature_engineering(df):
    # Create new features
    df['price_per_sqm'] = df['resale_price'] / df['floor_area_sqm']

    # Create LabelEncoders
    encoders = {}
    categorical_columns = ['town', 'flat_type', 'flat_model']

    # Encode categorical variables and store encoders
    for column in categorical_columns:
        le = LabelEncoder()
        df[f'{column}_encoded'] = le.fit_transform(df[column])
        encoders[column] = le

    # Save encoders dictionary
    with open('encoders.pkl', 'wb') as f:
        pickle.dump(encoders, f)

    return df, encoders
def save_processed_data(df):
    # Select features for the final dataset
    processed_columns = [
        'town_encoded', 'flat_type_encoded', 'flat_model_encoded',
        'floor_area_sqm', 'storey_median', 'flat_age', 'price_per_sqm',
        'resale_price'
    ]

    # Save processed features to CSV
    df[processed_columns].to_csv('processed_data.csv', index=False)

    # Also save original categorical mappings for reference
    mappings = pd.DataFrame({
        'town': df['town'].unique(),
        'town_encoded': [df[df['town'] == t]['town_encoded'].iloc[0] for t in df['town'].unique()]
    })
    mappings.to_csv('categorical_mappings.csv', index=False)
if __name__ == "__main__":
    # Load data
    print("Loading data...")
    df = load_data()

    # Clean data
    print("Cleaning data...")
    df_cleaned = clean_data(df)

    # Feature engineering
    print("Performing feature engineering...")
    df_engineered, encoders = feature_engineering(df_cleaned)

    # Save processed data
    print("Saving processed data...")
    save_processed_data(df_engineered)

    print("Data preprocessing completed! Files saved: processed_data.csv, encoders.pkl, categorical_mappings.csv")

Loading data...
Dataset shape: (196686, 11)
Cleaning data...


<ipython-input-9-8cd8400b35a4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = pd.to_datetime(df['month'])
<ipython-input-9-8cd8400b35a4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['flat_age'] = df['month'].dt.year - df['lease_commence_date']
<ipython-input-9-8cd8400b35a4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

Performing feature engineering...
Saving processed data...
Data preprocessing completed! Files saved: processed_data.csv, encoders.pkl, categorical_mappings.csv
